In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import wandb

from ssm import *
from S4 import *

In [2]:
S4Layer = cloneLayer(S4Layer)